<a href="https://colab.research.google.com/github/mathewpolonsky/Marketplace-Item-Matching/blob/main/Adding_Text_Difference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Находим разницу в текстах различных товаров

Идея в том, чтобы обучать модель только на разнице в товарах, так как одинаковые атрибуты не приносят большого количества информации.

Для этого используем библиотеку `difflib`, которая предоставляет разницу, в таком же виде, как например на GitHub.

In [ ]:
# download dataset 
!gdown 1-vKglR5qwkmXGT8EK2La2eiNy1ApKLPD
!unzip merged_datasets4nlp.zip
!mv merged_datasets4nlp datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

---

In [ ]:
from difflib import Differ
import re

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

In [ ]:
train = pd.read_parquet("datasets/merged_train4nlp.parquet")
test = pd.read_parquet("datasets/merged_test4nlp.parquet")

In [ ]:
differ = Differ()

texts_diff = []

for num in tqdm(range(train.shape[0])):
    text1, text2 = train.iloc[num][["full_text_1", "full_text_2"]].values
    
    # Убираем повторяющиеся \n
    text1, text2 = re.sub(r"\n+", '\n', text1), re.sub(r"\n+", '\n', text2)

    text1, text2 = text1.split('\n'), text2.split('\n')
    
    # Убираем повторяющиеся пробелы
    text1 = [re.sub(r'\s+', ' ', text).strip() for text in text1]
    text2 = [re.sub(r'\s+', ' ', text).strip() for text in text2]
    
    # Находим разницу
    # Сортируем тексты для коректного нахождения разницы
    # чтобы одинаковые отрибуты сравнивались друг с другом
    diff = differ.compare(sorted(text1), sorted(text2))
    
    # Объеденяем обратно тексты и помечаем к какому товару принадлежат атрибуты
    new_text = "\n".join(text for text in diff if text[0] in ['+', '-'])
    new_text = new_text.replace('- ', "Товар 1 | ").replace('+ ', "Товар 2 | ")
    
    # Если разницы нет
    if not new_text:
        new_text = "Товар 1 и Товар 2 одинаковы"

    texts_diff.append(new_text)

In [ ]:
train["text_difference"] = texts_diff

In [ ]:
train

,target,variantid1,variantid2,full_text_1,full_text_2,text_difference
0,0.0,51197862,51198054,name: Удлинитель TDM Electric Люкс УЛ05В 5 м (...,name: Удлинитель TDM Electric Люкс УЛ05В 1.5 м...,Товар 1 | name: Удлинитель TDM Electric Люкс У...
1,0.0,51197862,51199884,name: Удлинитель TDM Electric Люкс УЛ05В 5 м (...,name: Удлинитель TDM Electric Люкс УЛ05В 3 м (...,Товар 1 | name: Удлинитель TDM Electric Люкс У...
2,1.0,53062686,536165289,name: Картридж лазерный Комус 729 (4370B002) ч...,name: Картридж лазерный Комус 729 (4368B002) п...,"Товар 1 | color: черный, чер\nТовар 2 | color:..."
3,1.0,53602615,587809782,name: Картридж лазерный Комус 729 (4368B002) п...,name: Картридж лазерный Комус 729 (4370B002) ч...,Товар 1 | color: пурпурный\nТовар 2 | color: ч...
4,1.0,53602615,615149925,name: Картридж лазерный Комус 729 (4368B002) п...,name: Картридж лазерный Комус 729 (4368B002) п...,Товар 1 | name: Картридж лазерный Комус 729 (4...
...,...,...,...,...,...,...
306535,0.0,810680230,820119986,"name: Комплект 4 шт, Картридж лазерный NV Prin...","name: Комплект 2 шт, Картридж лазерный NV Prin...","Товар 1 | name: Комплект 4 шт, Картридж лазерн..."
306536,0.0,812434186,815345877,name: Для iPhone 7 плюс 8 плюс военная броня г...,name: Для iPhone 7 8 военная броня корпус гибр...,Товар 1 | color: желтый\nТовар 2 | color: сере...
306537,0.0,815637954,817550808,name: Карта памяти 64 ГБ\ncat3: Карты памяти и...,name: Карта памяти 1 ТБ\ncat3: Карты памяти и ...,Товар 1 | name: Карта памяти 64 ГБ\nТовар 2 | ...
306538,0.0,817327230,822083612,"name: Смартфон Mate48 Pro.. 8/256 ГБ, зеленый\...","name: Смартфон Mate48 Pro.. 10/512 ГБ, зеленый...",Товар 1 | name: Смартфон Mate48 Pro.. 8/256 ГБ...


In [ ]:
train.to_parquet("drive/MyDrive/datasets/merged_train4nlp_w_diff.parquet", engine='pyarrow', index=False)

In [ ]:
differ = Differ()

texts_diff = []

for num in tqdm(range(test.shape[0])):
    text1, text2 = test.iloc[num][["full_text_1", "full_text_2"]].values
    
    # Убираем повторяющиеся \n
    text1, text2 = re.sub(r"\n+", '\n', text1), re.sub(r"\n+", '\n', text2)

    text1, text2 = text1.split('\n'), text2.split('\n')
    
    # Убираем повторяющиеся пробелы
    text1 = [re.sub(r'\s+', ' ', text).strip() for text in text1]
    text2 = [re.sub(r'\s+', ' ', text).strip() for text in text2]
    
    # Находим разницу
    # Сортируем тексты для коректного нахождения разницы
    # чтобы одинаковые отрибуты сравнивались друг с другом
    diff = differ.compare(sorted(text1), sorted(text2))

    # Объеденяем обратно тексты и помечаем к какому товару принадлежат атрибуты
    new_text = "\n".join(text for text in diff if text[0] in ['+', '-'])
    new_text = new_text.replace('- ', "Товар 1 | ").replace('+ ', "Товар 2 | ")
    
    # Если разницы нет
    if not new_text:
        new_text = "Товар 1 и Товар 2 одинаковы"

    texts_diff.append(new_text)

  0%|          | 0/18084 [00:00<?, ?it/s]

In [ ]:
test["text_difference"] = texts_diff

In [ ]:
test

,variantid1,variantid2,full_text_1,full_text_2,text_difference
0,52076340,290590137,name: Батарейка AAA щелочная Perfeo LR03/10BL ...,name: Батарейка AAA щелочная Perfeo LR03/2BL m...,Товар 1 | name: Батарейка AAA щелочная Perfeo ...
1,64525522,204128919,"name: Смартфон Ulefone Armor X5 3/32 ГБ, черны...","name: Смартфон Ulefone Armor X3 2/32 ГБ, черны...",Товар 1 | name: Смартфон Ulefone Armor X5 3/32...
2,77243372,479860557,name: Цифровой кабель TV-COM HDMI 1.4 (M/ M) F...,name: Кабель HDMI 1.4 (Male/Male) (CG150S-1.5...,Товар 1 | name: Цифровой кабель TV-COM HDMI 1....
3,86065820,540678372,name: Игровая мышь проводная A4Tech Bloody P93...,"name: Мышь A4Tech Bloody P93s Bullet, серый, о...",Товар 1 | color: черный\nТовар 2 | color: серы...
4,91566575,258840506,"name: Смартфон Vivo Y93 1815 3/32 ГБ, черный\n...","name: Смартфон Vivo Y81 3/32 ГБ, черный\ncolor...","Товар 1 | color: черный, синий\nТовар 2 | colo..."
...,...,...,...,...,...
18079,666998614,667074522,name: Оперативная память Corsair VENGEANCE LPX...,name: Оперативная память Corsair VENGEANCE LPX...,Товар 1 | color: black\nТовар 1 | name: Операт...
18080,670036240,670048449,name: Зарядное устройство для HP 15-bw661ur бл...,name: Зарядное устройство для HP 15s-fq2066ur ...,Товар 1 | name: Зарядное устройство для HP 15-...
18081,670284509,684323809,name: Ремешок из натуральной кожи 20 мм для Sa...,name: Ремешок для часов UTHAI Z08 plus из нату...,"Товар 1 | color: желтый, красный\nТовар 2 | co..."
18082,692172005,704805270,name: Лазерный новогодний проектор для дома и ...,name: Новогодний проектор гирлянда морозостойк...,Товар 1 | color: синий\nТовар 2 | color: зелен...


In [ ]:
test.to_parquet("drive/MyDrive/datasets/merged_test4nlp_w_diff.parquet", engine='pyarrow', index=False)